Zaimplementuj aplikację szacującą czas ukończenia półmaratonu dla zadanych danych

1. Umieść dane w Digital Ocean Spaces

1. Napisz notebook, który będzie Twoim pipelinem do trenowania modelu
    * czyta dane z Digital Ocean Spaces
    * czyści je
    * trenuje model (dobierz odpowiednie metryki [feature selection])
    * nowa wersja modelu jest zapisywana lokalnie i do Digital Ocean Spaces

1. Aplikacja
    * opakuj model w aplikację streamlit
    * wdróż (deploy) aplikację za pomocą Digital Ocean AppPlatform 
    * wejściem jest pole tekstowe, w którym użytkownik się przedstawia, mówi o tym
    jaka jest jego płeć, wiek i tempo na 5km
    * jeśli użytkownik podał za mało danych, wyświetl informację o tym jakich danych brakuje
    * za pomocą LLM (OpenAI) wyłuskaj potrzebne dane, potrzebne dla Twojego modelu
    do określenia, do słownika (dictionary lub JSON)
    * tę część podepnij do Langfuse, aby zbierać metryki o skuteczności działania LLM'a



In [1]:
import pandas as pd
from pycaret.regression import *
from sklearn.model_selection import train_test_split
from datetime import datetime

# Wczytanie danych
df_2023 = pd.read_csv('halfmarathon_wroclaw_2023__final.csv', sep=';')
df_2024 = pd.read_csv('halfmarathon_wroclaw_2024__final.csv', sep=';')


# Obliczenie wieku na podstawie roku urodzenia
df_2023['Wiek'] = 2023 - df_2023['Rocznik']
df_2024['Wiek'] = 2024 - df_2024['Rocznik']

# Połączenie obu DataFrame w jeden
df = pd.concat([df_2023, df_2024], ignore_index=True)

# Usunięcie wierszy z brakującymi danymi w istotnych kolumnach
df = df[df['5 km Czas'].notna()]
df = df[df['10 km Czas'].notna()]
df = df[df['15 km Czas'].notna()]
df = df[df['20 km Czas'].notna()]
df = df[df['Czas'].notna()]

# Mapowanie płci (0 = K, 1 = M)
df['Płeć'] = df['Płeć'].map({'K': 0, 'M': 1})

# Kolumny do usunięcia
columns_to_drop = ['Miejsce', 'Numer startowy', 'Rocznik', 'Kategoria wiekowa', 'Imię', 'Nazwisko', 'Miasto', 'Kraj', 'Drużyna', 'Płeć Miejsce', 'Kategoria wiekowa Miejsce', 
                  '5 km Miejsce Open', '10 km Miejsce Open', '15 km Miejsce Open', '20 km Miejsce Open', 'Tempo Stabilność']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Funkcje konwertujące czas i tempo na sekundy
def convert_time_to_seconds(time_str):
    if pd.isna(time_str): return None
    time_parts = time_str.split(':')
    if len(time_parts) == 2:
        minutes, seconds = map(int, time_parts)
        return minutes * 60 + seconds
    elif len(time_parts) == 3:
        hours, minutes, seconds = map(int, time_parts)
        return hours * 3600 + minutes * 60 + seconds
    return None

# Zastosowanie konwersji
df['Czas_półmaratonu'] = df['Czas'].apply(convert_time_to_seconds)
df['Czas_na_5km'] = df['5 km Czas'].apply(convert_time_to_seconds)
df['Czas_na_10km'] = df['10 km Czas'].apply(convert_time_to_seconds)
df['Czas_na_15km'] = df['15 km Czas'].apply(convert_time_to_seconds)

# Usunięcie niepotrzebnych kolumn
df.drop(columns=['Czas', 'Tempo', '5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo'], inplace=True, errors='ignore')

df

# # Funkcja do trenowania modelu i przewidywania czasów na różnych odcinkach
# def train_and_predict(target_column):
#     # Podział danych na zbiór treningowy i testowy
#     train_data, test_data = train_test_split(df, test_size=0.2, random_state=123)

#     # Ustawienie środowiska PyCaret z dodatkowymi cechami (w tym Płeć, Wiek)
#     setup(data=train_data,
#           target=target_column,
#           session_id=123, 
#           normalize=True,
#           feature_selection=True,
#           remove_multicollinearity=True
#         )

#     # Porównanie modeli
#     best_model = compare_models(fold=5)

#     # Finalizacja najlepszego modelu
#     final_model = finalize_model(best_model)

#     # Zapisanie najlepszego modelu
#     save_model(final_model, f'{target_column}_model')

#     return final_model

# # Trenowanie i zapisanie modeli dla różnych celów (czas na 5 km, 10 km, 15 km i pełny półmaraton)
# halfmarathon_time_model = train_and_predict('Czas_półmaratonu')

,Płeć,Wiek,Czas_półmaratonu,Czas_na_5km,Czas_na_10km,Czas_na_15km
0,1,31.0,3899,877,1755,2687
1,1,37.0,3983,888,1783,2726
2,1,27.0,4104,946,1883,2854
3,1,35.0,4216,971,1930,2929
4,1,28.0,4227,972,1955,2971
...,...,...,...,...,...,...
19244,0,47.0,11866,2396,5056,7858
19246,0,46.0,12068,2526,5254,8128
19247,0,69.0,12138,2960,5899,8602
19248,0,43.0,12512,2808,5769,8763
